In [ ]:
from enum import Enum
from typing import Union

class Scope :
    def __init__(self):
        self.var = {}
    def get(self, var):
        return self.var.get(var)
    def set(self, var, value):
        self.var[var] = value

In [26]:
# Defining boolean instructions for boolean trees
class binstr(Enum):
    AND = 1
    OR = 2
    NOT = 3

class bexp:

    left = Union[bool, 'bexp', str]
    instr = binstr
    right = 'bexp'

    def __init__(self, left: Union[bool, 'bexp', str], instr: binstr = None, right: 'bexp' = None):
        self.left = left
        self.right = right
        self.instr = instr

    def evalB(self, scope:Scope = None) -> bool:
        '''Evaluates the bexp'''
        match self.instr:
            case None:
                if(isinstance(self.left, str)):
                    if(scope != None):
                        return scope.get(self.left)
                    else:
                        print('Err : scope null')
                        exit(1)
                return self.left
            case binstr.AND:
                l = self.left.evalB(scope)
                r = self.right.evalB(scope)
                return (l and r)
            case binstr.OR:
                l = self.left.evalB(scope)
                r = self.right.evalB(scope)
                return (l or r)
            case binstr.NOT:
                l = self.left.evalB(scope)
                return (not l)
            case _:
                print("Err : Wrong binstr")
                exit(1)
    
    def __str__(self) -> str:
        match self.instr:
            case None:
                return str(self.left)
            case binstr.AND:
                l = str(self.left)
                r = str(self.right)
                return ('('+l + ' and ' + r+')')
            case binstr.OR:
                l = str(self.left)
                r = str(self.right)
                return ('('+l + ' or ' + r+')')
            case binstr.NOT:
                l = str(self.left)
                return ('(not '+ l+')')
            case _:
                print("Err : Wrong binstr")
                exit(1)

In [27]:
# first tests with basic boolean operations without any variable
eT = bexp(True)
eF = bexp(False)
eA = bexp(eT, binstr.AND, eF)
eO = bexp(eF, binstr.OR, eT)
eN = bexp(eF, binstr.NOT)
eAA = bexp(eO, binstr.AND, eT)

testTree1 = [eT, eF, eA, eO, eN, eAA]

for tree in testTree1:
    print(str(tree)+' vaut '+str(tree.evalB()))

True vaut True
False vaut False
(True and False) vaut False
(False or True) vaut True
(not False) vaut True
((False or True) and True) vaut True


In [29]:
# tests of basic boolean operations with variables
scope = Scope()
scope.set('t',True)
scope.set('f' ,False)

eTS = bexp('t')
eFS = bexp('f')
eAS = bexp(eTS, binstr.AND, eFS)
eOS = bexp(eFS, binstr.OR, eTS)
eNS = bexp(eFS, binstr.NOT)
eAAS = bexp(eOS, binstr.AND, eTS)

testTree2 = [eTS, eFS, eAS, eOS, eNS, eAAS]

for tree in testTree2:
    print(str(tree)+' vaut '+str(tree.evalB(scope)))

t vaut True
f vaut False
(t and f) vaut False
(f or t) vaut True
(not f) vaut True
((f or t) and t) vaut True


In [42]:
# Defining operation instruction for mathematical operation tree
class opinstr(Enum):
    PLUS = 1
    MINUS = 2
    DIV = 3
    MULT = 4
    MOD = 5
    POW  = 6
    VALUE = 7

class opexp():

    instrType = opinstr
    left = Union['opexp', int, float, str]
    right = 'opexp'

    def __init__(self, instrType:opinstr, left:Union['opexp', int, bool, float, str], right:'opexp'=None) -> None:
        self.instrType = instrType
        self.left = left
        self.right = right
        if(self.right==None and instrType!=opinstr.VALUE):
            print('Err : no right operation')
            exit(1)
    
    def evalO(self, scope:Scope) -> Union[int, float, str] :
        l = self.left.evalO(scope)
        if(self.instrType!=opinstr.VALUE):
            r = self.right.evalO(scope)
        if(self.instrType!=opinstr.PLUS):
            if(isinstance(l, str) or isinstance(r, str)):
                print('Err : can\'t eval this with strings')
                exit(1)
        match self.instrType:
            case opinstr.PLUS:
                return (l+r)
            case opinstr.MINUS:
                return (l-r)
            case opinstr.DIV:
                return (l/r)
            case opinstr.MULT:
                return (l*r)
            case opinstr.MOD:
                return (l % r)
            case opinstr.POW:
                return (l**r)
            case opinstr.VALUE:
                if(isinstance(self.left, str)):
                    return scope.get(self.left)
                else:
                    return self.left
    
    def __str__(self) -> str:
        l = self.left.evalO(scope)
        if(self.instrType!=opinstr.VALUE):
            r = self.right.evalO(scope)
        match self.instrType:
            case opinstr.PLUS:
                return ('('+l+' + '+r+')')
            case opinstr.MINUS:
                return ('('+l+' - '+r+')')
            case opinstr.DIV:
                return ('('+l+' / '+r+')')
            case opinstr.MULT:
                return ('('+l+' * '+r+')')
            case opinstr.MOD:
                return ('('+l+' mod '+r+')')
            case opinstr.POW:
                return ('('+l+'^'+r+')')
            case opinstr.VALUE:
                return str(self.left)

In [43]:
# first tests with basic mathematical operations without any variable
e1 = opexp(opinstr.VALUE, 1)
e2 = opexp(opinstr.VALUE, 2)
e3 = opexp(opinstr.VALUE, 3)
es1 = opexp(opinstr.VALUE, 'string1')
es2 = opexp(opinstr.VALUE, 'string2')

ep1 = opexp(opinstr.PLUS, e1, e2)
ep2 = opexp(opinstr.PLUS, es1, es2)
em1 = opexp(opinstr.MINUS, e2, e1)
em2 = opexp(opinstr.MINUS, es2, es1)
ed1 = opexp(opinstr.DIV, e2, e1)
ed2 = opexp(opinstr.DIV, es2, es1)
emu1 = opexp(opinstr.MULT, e2, e1)
emu2 = opexp(opinstr.MULT, es2, es1)
emo1 = opexp(opinstr.MOD, e2, e1)
emo2 = opexp(opinstr.MOD, es2, es1)
epo1 = opexp(opinstr.POW, e2, e1)
epo2 = opexp(opinstr.POW, es2, es1)

testtree3 = [ep1, ep2, em1, em2, ed1, ed2, emu1, emu2, emo1, emo2, epo1, epo2]
for tree in testtree3:
    print(str(tree)+' vaut '+str(tree.evalO(scope)))

AttributeError: 'int' object has no attribute 'evalO'

In [35]:
class instr(Enum):
    WHILE = 1
    ASSIGN = 2
    IF = 3
    SEQ = 4
    SKIP = 5

class exp():

    insType = instr
    cond = bexp
    inst1 = 'exp'
    inst2 = 'exp'
    var = str
    # value TODO

    def __init__(self, insType:instr, cond:bexp=None, inst1:bexp=None, inst2:bexp=None, var:str=None, value=None) -> None:
        self.insType = insType
        match insType:
            case instr.WHILE:
                self.initWHILE(cond, inst1)
            case instr.ASSIGN:
                self.initASSIGN(var, value)
            case instr.IF:
                self.initIF(cond, inst1, inst2)
            case instr.SEQ:
                self.initSEQ(inst1, inst2)
            case instr.SKIP:
                pass
            case _:
                print('Err : invalid insType')
                exit(1)

    
    def initWHILE(self, cond:bexp, inst:'exp') -> None:
        if(cond==None or inst==None):
            print('Err : variable None in initWHILE')
            exit(1)
        self.cond = cond
        self.inst1 = inst
    
    def initASSIGN(self, var:str, value = None) -> None:
        if(var==None or value==None):
            print('Err : variable None in initASSIGN')
            exit(1)
        self.var = var
        self.value = value
    
    def initIF(self, cond:bexp, inst1:'exp', inst2:'exp') -> None:
        if(cond==None or inst1==None or inst2==None):
            print('Err : variable None in initIF')
            exit(1)
        self.cond = cond
        self.inst1 = inst1
        self.inst2 = inst2
    
    def initSEQ(self, inst1:'exp', inst2:'exp') -> None:
        if(inst1==None or inst2==None):
            print('Err : variable None in initWHILE')
            exit(1)
        self.inst1 = inst1
        self.inst2 = inst2


